In [ ]:
try:
    from undi import MuonNuclearInteraction
except (ImportError, ModuleNotFoundError):
    import sys
    sys.path.append('../undi')
    from undi import MuonNuclearInteraction
import matplotlib.pyplot as plt
import numpy as np

# Define lattice structure

The octahedral muon site in Cu is defined below.

In [ ]:
angtom=1.0e-10 # m
a=3.6212625504 # Cu lattice constant, in Angstrom

Cu_Quadrupole_moment =  (-0.211) * (10**-28) # m^-2
atoms = [

    {'Position': np.array([0.5, 0.5, 0.5]) * a * angtom,
     'Label': 'mu'},

    {'Position': np.array([0.0    ,    0.5   ,     0.5])*angtom*a,
     'Label': 'Cu',
     'ElectricQuadrupoleMoment': Cu_Quadrupole_moment,
     # 'OmegaQmu': 3.2e6 # s^-1
    },

    {'Position': np.array([0.5    ,    0.0   ,     0.5])*angtom*a,
     'Label': 'Cu',
     'ElectricQuadrupoleMoment': Cu_Quadrupole_moment,
     # 'OmegaQmu': 3.2e6 # s^-1
    },

    {'Position': np.array([1.0    ,   0.5   ,     0.5])*angtom*a,
     'Label': 'Cu',
     'ElectricQuadrupoleMoment': Cu_Quadrupole_moment,
     # 'OmegaQmu': 3.2e6 # s^-1
    },

    {'Position': np.array([0.5    ,    1.0   ,     0.5])*angtom*a,
     'Label': 'Cu',
     'ElectricQuadrupoleMoment': Cu_Quadrupole_moment,
     # 'OmegaQmu': 3.2e6 # s^-1
    },

    {'Position': np.array([0.5    ,    0.5   ,     0.0])*angtom*a,
    'Label': 'Cu',
    'ElectricQuadrupoleMoment': Cu_Quadrupole_moment,
    # 'OmegaQmu': 3.2e6 # s^-1
    },

    {'Position': np.array([0.5    ,    0.5   ,     1.0])*angtom*a,
     'Label': 'Cu',
     'ElectricQuadrupoleMoment': Cu_Quadrupole_moment,
     # 'OmegaQmu': 3.2e6 # s^-1
    }
]

# Set Electric Field Gradient

There are two ways of treating quadrupole coupling with undi.
One can either specify 'OmegaQmu' and assume the electric field
gradient generated by the muon, or explicitly set it.
We opt for the latter option here. You may get the same result
removing the comment from the lines defining `OmegaQmu` above.

In [ ]:
elementary_charge=1.6021766E-19 # Coulomb = ampere ⋅ second

def Vzz_for_unit_charge_at_distance(r):
    epsilon0 = 8.8541878E-12 # ampere^2 ⋅ kilogram^−1 ⋅ meter^−3 ⋅ second^4
    elementary_charge=1.6021766E-19 # Coulomb = ampere ⋅ second
    Vzz = (2./(4 * np.pi * epsilon0)) * (elementary_charge / (r**3))
    return Vzz

def Vzz_from_Celio_PRL():
    # 0.27 angstrom^−3 is from PRL 39 836
    # (4 pi epsilon_0)^−1 (0.27 angstrom^−3) elementary_charge = 3.8879043E20 meter^−2 ⋅ volts
    Vzz = 1.02702 * 3.8879043E20 # the factor 1.02702 gives exactly 3.2e6 s^-1 for omega_q
    return Vzz

def get_V(p_mu, p_N, Vzz):
    x=p_N-p_mu
    n = np.linalg.norm(x)
    x /= n; r = 1. # keeping formula below for clarity
    return -Vzz * ( (3.*np.outer(x,x)-np.eye(3)*(r**2))/r**5 )


for idx, atom in enumerate(atoms):
    if atom['Label'] == 'Cu':
        atoms[idx]['EFGTensor'] = get_V(atoms[0]['Position'], atom['Position'], Vzz_from_Celio_PRL())

# Polarization function

The muon polarization is obtained with the method introduced by Celio.

In [ ]:
steps = 100
tlist = np.linspace(0, 16e-6, steps)
signals = np.zeros([6,steps], dtype=np.float)


LongitudinalFields = (0.0, 0.001, 0.003, 0.007, 0.008, 0.01)
for idx, Bmod in enumerate(LongitudinalFields):

    # Put field along muon polarization, that is always z
    B = Bmod * np.array([0,0.,1.])
    NS = MuonNuclearInteraction(atoms, external_field=B, log_level='info')

    # rotate the sumple such that the muon spin is aligned with
    # the 111 direction (and, just for convenience, the muon position is
    # set to (0,0,0) )
    NS.translate_rotate_sample_vec(np.array([1.,1.,1.]))

    print("Computing signal 4 times with LF {} T...".format(Bmod), end='', flush=True)
    signal_Cu = NS.celio(tlist,  k=6)
    for i in range(3):
        print('{}...'.format(i+1), end='', flush=True)
        signal_Cu += NS.celio(tlist, k=6)
    print('done!')
    signal_Cu /= float(i+1+1)
    del NS

    signals[idx]=signal_Cu

...and the results is:

In [ ]:
fig, axes = plt.subplots(1,1)
for i, Bmod in enumerate(LongitudinalFields):
    color = list(np.random.choice(range(256), size=3)/256)
    axes.plot(tlist, signals[i], label='{} G'.format(Bmod*1e4), linestyle='-', color=color)
axes.set_ylim((-0.1,1.1))
plt.legend()
plt.show()